## Importing important packages:

In [1]:
# Based on Pytorch tutorial for transfer learning
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # interactive mode

## Model Loading:

In [2]:
# Define your model architecture
model = models.resnet34(pretrained=True)

# Replace the last fc layer with a new one that has 2 output units
model.fc = nn.Linear(512, 7)

# Load the saved model's state dictionary
model.load_state_dict(torch.load('models/model.pt'))

# Set the model to evaluation mode
model.eval()

c:\Users\LAPTOP WORLD\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\LAPTOP WORLD\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

This code defines a ResNet-34 architecture model pre-trained on ImageNet dataset, then replaces the last fully connected layer (fc) with a new one that has 2 output units. It then loads the saved state dictionary of the model from the file "model.pt". Finally, it sets the model to evaluation mode using the `eval()` method.

The `models.resnet34(pretrained=True)` line creates the ResNet-34 architecture pre-trained on the ImageNet dataset.

The `model.fc = nn.Linear(512, 2)` line replaces the last fully connected layer of the model with a new one having 7 output units.

The `model.load_state_dict(torch.load('models/model.pt'))` line loads the saved state dictionary of the model from the file "model.pt" located in the "models" directory.

The `model.eval()` line sets the model to evaluation mode by turning off the dropout and batch normalization layers, which are commonly used during training, and enabling the batch statistics of these layers to be fixed. This ensures that the output produced by the model is deterministic and not affected by randomness that is introduced during training.

## Data Reading:

In [3]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomApply(torch.nn.ModuleList([
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.1,0.1,0.1,0.1),
        transforms.RandomAffine(5)]), p=0.5),
        transforms.RandomPosterize(bits=2),
        transforms.GaussianBlur(kernel_size=5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Download dataset for candidate testing from https://download.pytorch.org/tutorial/hymenoptera_data.zip
data_dir = ''
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'validation']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1,
                                             shuffle=False, num_workers=2)
              for x in ['train', 'validation']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation']}
class_names = image_datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

This code defines data transformations for training and validation datasets, and then loads the Hymenoptera dataset for training and validation. It also creates dataloaders for the training and validation sets, and sets the device to use for computation.

The data transformation is defined using the `transforms.Compose` method from the `torchvision.transforms` module. The transformations applied to the training dataset include resizing the image to 256x256 pixels, cropping the image to 224x224 pixels at the center, randomly applying data augmentation techniques such as horizontal flipping, color jittering, and affine transformation, applying a random posterization filter with 2 bits, applying a Gaussian blur filter with kernel size 5, converting the image to a PyTorch tensor, and normalizing the image with the mean and standard deviation of the ImageNet dataset. The transformations applied to the validation dataset include resizing and cropping the image, converting the image to a PyTorch tensor, and normalizing the image with the mean and standard deviation of the ImageNet dataset.

The Hymenoptera dataset is loaded using the `datasets.ImageFolder` method from the `torchvision.datasets` module. The dataset is downloaded from the URL provided in the code, and is split into training and validation sets. The dataloaders are created using the `torch.utils.data.DataLoader` method from the `torch.utils.data` module, and batch size is set to 6. The `shuffle` parameter is set to `False` for the validation dataloader to ensure that images are loaded in a fixed order.

The `dataset_sizes` variable is a dictionary that contains the number of images in each dataset, and the `class_names` variable contains the class names for the dataset. Finally, the `device` variable is set to use the GPU if it is available, and the CPU otherwise.

In [4]:
image_datasets['train'].classes

['bowriding',
 'cabincruiser',
 'commercial',
 'enforcement',
 'halfcab',
 'humanpowered',
 'open']

## Predicting:

In [5]:
import pandas as pd

model.eval()
predictions = []
true_labels = []
confidences = []

with torch.no_grad():
    for inputs, labels in dataloaders['validation']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        confidence = torch.nn.functional.softmax(outputs, dim=1)

        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
        confidences.append(confidence.cpu().numpy().max())
        print('-',end='')
        
df =  pd.DataFrame({
    'Image Name': dataloaders['validation'].dataset.samples,
    'Real Label': true_labels,
    'Predicted Label': predictions,
    'Model Confidence': confidences
})

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

This code is performing inference on a PyTorch model using the validation dataset and storing the results in a Pandas DataFrame.

First, the model is set to evaluation mode using `model.eval()`. Then, a loop is started where the model predicts labels for each batch of validation data. For each batch, the inputs and labels are moved to the device specified in the `device` variable. The model's `outputs` are generated by passing the inputs through the model, and the predicted labels are obtained by taking the index of the maximum value of each output. Additionally, the confidence of the model's prediction is obtained by passing the outputs through a softmax function and taking the maximum value. 

The predicted labels, true labels, and model confidences are stored in Python lists `predictions`, `true_labels`, and `confidences`, respectively. The lists are then concatenated together using `extend`. The `print('-',end='')` statement is used to show a progress bar while the loop is running.

Finally, a Pandas DataFrame is created using the `pd.DataFrame` constructor. The `Image Name` column contains the filenames of the validation images, the `Real Label` column contains the true labels for each image, the `Predicted Label` column contains the model's predicted labels for each image, and the `Model Confidence` column contains the confidence of the model's prediction for each image. The DataFrame is returned and can be used for further analysis or evaluation.

In [6]:
label_dct = {
    'Bowriding': 0,
    'CabinCruise': 1,
    'Commercial': 2,
    'Enforcement': 3,
    'HalfCab': 4,
    'HumanPowered': 5,
    'Open': 6,
}

In [7]:
for k,v in label_dct.items():
    df['Real Label'] [df['Real Label'] == v]= k
    df['Predicted Label'][df['Predicted Label'] == v] = k

C:\Users\LAPTOP WORLD\AppData\Local\Temp\ipykernel_8788\3337473421.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Real Label'] [df['Real Label'] == v]= k
C:\Users\LAPTOP WORLD\AppData\Local\Temp\ipykernel_8788\3337473421.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Predicted Label'][df['Predicted Label'] == v] = k


In [8]:
df.to_csv('predictions.csv')

In [9]:
df.tail(40)

,Image Name,Real Label,Predicted Label,Model Confidence
394,(validation\open\Open-415699f2-09e9-4b1d-9989-...,Open,Open,0.975402
395,(validation\open\Open-4834b65c-32ec-4a0c-8666-...,Open,Open,0.972710
396,(validation\open\Open-4a9fbd6b-0b6a-43e7-859f-...,Open,Open,0.993735
397,(validation\open\Open-54c0dbbf-00fe-465e-b5d8-...,Open,Open,0.949913
398,(validation\open\Open-58b586f8-7fc2-4b5c-b011-...,Open,Open,0.863987
399,(validation\open\Open-5a17d68b-3466-4d76-8cd0-...,Open,Open,0.978697
400,(validation\open\Open-5cea27a4-6344-4fa0-aabc-...,Open,Open,0.980342
401,(validation\open\Open-60e88e8f-93fa-43a0-b7f3-...,Open,Open,0.982651
402,(validation\open\Open-65c014ec-9338-4d0c-8e25-...,Open,Open,0.976694
403,(validation\open\Open-6d545b08-1c6f-4662-bb2d-...,Open,Open,0.977828


Let's show the pictures that the model predicted wrong:

In [10]:
df[df.iloc[:,1]!=df.iloc[:,2]]

,Image Name,Real Label,Predicted Label,Model Confidence
8,(validation\bowriding\Bowriding-17da69ed-b393-...,Bowriding,HumanPowered,0.915015
31,(validation\bowriding\Bowriding-4ff45269-a76c-...,Bowriding,HumanPowered,0.834029
35,(validation\bowriding\Bowriding-5d773c78-e166-...,Bowriding,CabinCruise,0.502064
42,(validation\bowriding\Bowriding-8d09ae88-5b3f-...,Bowriding,CabinCruise,0.643526
43,(validation\bowriding\Bowriding-8ddbe737-0b9c-...,Bowriding,HumanPowered,0.838397
57,(validation\bowriding\Bowriding-cf271996-185d-...,Bowriding,HumanPowered,0.876554
68,(validation\bowriding\Bowriding-dcd6ec0a-8891-...,Bowriding,CabinCruise,0.506464
132,(validation\cabincruiser\CabinCruiser-f238fe14...,CabinCruise,Commercial,0.648547
261,(validation\halfcab\HalfCab-0ed4a851-2cd6-4f72...,HalfCab,Open,0.628680
267,(validation\halfcab\HalfCab-1a594a18-ee2b-412e...,HalfCab,Open,0.929733


#### it seems like the model find it the hardist to predict Bowriding.

In [11]:
from sklearn.metrics import accuracy_score

accuracy_score(df.iloc[:,1],df.iloc[:,2])

0.9723502304147466

: 

### Finally the model got an accuracy of 97.24% on the testing set.